In [ ]:
from google.colab import files

files.upload_file('intents.json')

In [1]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=85273236f02fb7313add8d0938aeb6f6f7305b0df79d9d900623c161fb509cfe
  Stored in directory: /root/.cache/pip/wheels/4a/d5/f8/9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


In [2]:
import nltk
nltk.download('punkt') # stop words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.stem.lancaster import LancasterStemmer
stem = LancasterStemmer() # stem of words instance


In [6]:
data = []

In [4]:
import numpy as np
import pandas as pd
import tflearn
import tensorflow
import pickle
import json
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
words = []
labels = []
X = []
Y = []

In [22]:
def create_component():
  for intent in data["intents"]:
    for pattern in intent["patterns"]:
      words_aux = nltk.word_tokenize(pattern)
      words.extend(words_aux)
      X.append(words_aux)
      Y.append(intent["tag"])

    if intent["tag"] not in labels:
      labels.append(intent["tag"])

create_component()

TypeError: ignored

In [23]:
words = [stem.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))

In [24]:
labels = sorted(labels)

In [10]:
training = []
output = []

In [12]:
out_empty = [0 for _ in range(len(labels))]

In [19]:
def create_stem_lem():
  for x, pattern in enumerate(X):
    bag = []

    wrds = [stem.stem(w) for w in pattern]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
    
    output_row = out_empty[:]
    output_row[labels.index(Y[x])] = 1

    training.append(bag)
    output.append(output_row)

create_stem_lem()


In [15]:
training = np.array(training)
output = np.array(output)

In [16]:
tensorflow.compat.v1.reset_default_graph()

In [21]:
def create_model():
  net = tflearn.input_data(shape=([None, len(training[0])]))
  net = tflearn.fully_connected(net, 8)
  net = tflearn.fully_connected(net, 8)
  net = tflearn.fully_connected(net, 8)
  net = tflearn.fully_connected(net, 8)
  net = tflearn.fully_connected(net, len(output[0], activation="softmax"))
  net = tflearn.regression(net)

  return tflearn.bidirectional_rnn(net)



model = create_model()

IndexError: ignored

In [18]:
def fit():
  model.fit(training, output, n_epoch=1000, batch_size=12, show_metric=True)
  model.save("chatbot.tflearn")

In [ ]:
try:
  model.load("chatbot.tflearn")
except:
  fit()

In [20]:
def create_bag_of_words(inp, words):
  bag = [0 for _ in range(len(words))]

  word_t = nltk.word_tokenize(inp)
  word_t = [stem.stem(word.lower()) for word in word_t]

  for se in word_t:
    for i, w in enumerate(words):
      if se == w:
        bag[i] = 1

  return np.array(bag)



In [ ]:
def chatBot(message):
  response = model.predict(create_bag_of_words(message))[0]
  idx = np.argmax(response)
  tag = labels[idx]

  if response[idx] > .5:
    for tg in data["intents"]:
      if tg["tag"] == tag:
        responses = tg["responses"]

        return random.choice(responses)
      
      else:
        return "I cannot understand you"

